In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv(r"./Aleppo2017_processed.csv",encoding='utf-8')
print(df.head())

    id                 time     gl
0  183  2015-05-16 05:35:41  162.0
1  183  2015-05-16 05:30:41  164.0
2  183  2015-05-16 05:25:41  168.0
3  183  2015-05-16 05:20:41  169.0
4  183  2015-05-16 05:15:41  170.0


In [2]:
ID = df['id'].value_counts().index
print(ID)

Index([263,  77, 193, 277, 229, 245, 155, 251, 111, 164,
       ...
       128, 264, 186, 162, 249,  39,  52, 266, 223, 289],
      dtype='int64', name='id', length=226)


# 将数据整合 

## 先将每个受试者的数据提取，安装时间排序再分段，每500点为一段

### for example  第一个受试者的数据窗口

### 第一窗[0,1,2,3,4,......499] target = 500
### 第二窗[1,2,3,4,5,......500] target = 501
### 第三窗[2,3,4,5,6,......501] target = 502
### 第四窗[3,4,5,6,7,......502] target = 503

#### 那data_feature 内的数据为：
[第一个受试者的数据：
[0,1,2,3,4,......499],    #targets:[500]
[1,2,3,4,5,......500],    #targets:[501]
[2,3,4,5,6,......501],    #targets:[502]
[3,4,5,6,7,......502],    #targets:[503]
第二个受试者的数据：
[0,1,2,3,4,......499],
[1,2,3,4,5,......500],
[2,3,4,5,6,......501],
[3,4,5,6,7,......502]]

#### 对每个窗口提取特征
[第一个受试者的数据：
[0,1,2,3,4,......499],    #feature:[均值, 标准差, 方差，协方差]  target = 500
[1,2,3,4,5,......500],    #targets:[均值, 标准差, 方差，协方差]  target = 501
[2,3,4,5,6,......501],    #targets:[均值, 标准差, 方差，协方差]  target = 502
[3,4,5,6,7,......502],    #targets:[均值, 标准差, 方差，协方差]  target = 503
第二个受试者的数据：
[0,1,2,3,4,......499],
[1,2,3,4,5,......500],
[2,3,4,5,6,......501],
[3,4,5,6,7,......502]]

In [44]:
data_feature = []
data_target = []
seq = 500
data = df[df['id'] == 263]
#将time改成Date类数据类型
data['time'] = pd.to_datetime(data['time'], format="%Y-%m-%d  %H:%M:%S")
#按照时间排序
data = data.sort_values(by='time')   
data['target'] = data['gl'].shift(-1)

data.dropna()                      # 使用了shift函数，在最后必然是有缺失值的，这里去掉缺失值所在行
data = data[['gl','target']].astype(np.float32)  # 修改数据类型

for index in range(len(data) - seq):
    # 构建特征集
    data_feature.append(data['gl'][index: index + seq].values)
    # 构建target集
    data_target.append(data['target'][index:index + seq])
print(len(data_feature))
   

C:\Users\admin\AppData\Local\Temp\ipykernel_5652\1675604409.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['time'] = pd.to_datetime(data['time'], format="%Y-%m-%d  %H:%M:%S")


94180


In [30]:
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
import numpy as np
import torch
data_path = r"./Aleppo2017_processed.csv"
class myData(Dataset):
    def __init__(self):
        self.df = pd.read_csv(data_path,encoding='utf-8')
        # 将所有被试的特征放入data_feature,将所有被试的标签放入data_targets
        self.all_data_feature = []
        self.all_data_target = []
        self.data_feature = []
        self.data_target = []
        self.ID = self.df['id'].value_counts().index
        seq = 1000    #这个也算超参数之一
        for i in self.ID:
            self.data = self.df[self.df['id'] == i].copy()
            self.data = self.data.sort_values(by='time')
            # 时间为行索引
            self.data.index = self.data['time']

            # 设置标签
            self.data['target'] = self.data['gl'].shift(-1)
            self.data.dropna()  # 使用了shift函数，在最后必然是有缺失值的，这里去掉缺失值所在行
            self.data = self.data[['gl', 'target']].astype(np.float32)  # 修改数据类型
            self.data = self.data[:16439]    #所有受试者中血糖数据个数最小 为16439

            # 划分数据窗口
           
            for index in range(len(self.data) - seq):
                # 构建特征集
                self.data_feature.append(self.data['gl'][index: index + seq].values)
                # 构建target集
                self.data_target.append(self.data['target'].iloc[index + seq])
            self.all_data_feature.append(self.data_feature)
            self.all_data_target.append(self.data_target)
            self.data_feature = []
            self.data_target = []
            print("id为 {}的受试者的数据窗口加载完成".format(i))
        self.all_data_feature = torch.tensor(self.all_data_feature)
        self.all_data_target = torch.tensor(self.all_data_target)
        

    def __getitem__(self,idx):
        datas = self.all_data_feature[idx]
        targets = self.all_data_target[idx]
        return datas, targets

    def __len__(self):
        return len(self.ID)

allData = myData()
# 自动调用__getitem__( )


id为 263的受试者的数据窗口加载完成
id为 77的受试者的数据窗口加载完成
id为 193的受试者的数据窗口加载完成
id为 277的受试者的数据窗口加载完成
id为 229的受试者的数据窗口加载完成
id为 245的受试者的数据窗口加载完成
id为 155的受试者的数据窗口加载完成
id为 251的受试者的数据窗口加载完成
id为 111的受试者的数据窗口加载完成
id为 164的受试者的数据窗口加载完成
id为 228的受试者的数据窗口加载完成
id为 227的受试者的数据窗口加载完成
id为 68的受试者的数据窗口加载完成
id为 130的受试者的数据窗口加载完成
id为 271的受试者的数据窗口加载完成
id为 23的受试者的数据窗口加载完成
id为 232的受试者的数据窗口加载完成
id为 95的受试者的数据窗口加载完成
id为 7的受试者的数据窗口加载完成
id为 137的受试者的数据窗口加载完成
id为 102的受试者的数据窗口加载完成
id为 70的受试者的数据窗口加载完成
id为 67的受试者的数据窗口加载完成
id为 101的受试者的数据窗口加载完成
id为 127的受试者的数据窗口加载完成
id为 269的受试者的数据窗口加载完成
id为 250的受试者的数据窗口加载完成
id为 243的受试者的数据窗口加载完成
id为 240的受试者的数据窗口加载完成
id为 220的受试者的数据窗口加载完成
id为 205的受试者的数据窗口加载完成
id为 17的受试者的数据窗口加载完成
id为 187的受试者的数据窗口加载完成
id为 253的受试者的数据窗口加载完成
id为 123的受试者的数据窗口加载完成
id为 135的受试者的数据窗口加载完成
id为 166的受试者的数据窗口加载完成
id为 214的受试者的数据窗口加载完成
id为 139的受试者的数据窗口加载完成
id为 198的受试者的数据窗口加载完成
id为 76的受试者的数据窗口加载完成
id为 138的受试者的数据窗口加载完成
id为 105的受试者的数据窗口加载完成
id为 146的受试者的数据窗口加载完成
id为 184的受试者的数据窗口加载完成
id为 65的受试者的数据窗口加载完成
id为 81的受试者的数据窗口加载完成
id为 109的受试者的数据窗口加载完成
id为 

In [31]:
len(allData)

226

In [33]:
datas,targets = allData[0]
print(datas.shape)
print(targets.shape)

torch.Size([15439, 1000])
torch.Size([15439])


In [34]:
data2, target2 = allData[1]
print(data2.shape)
print(target2.shape)
print(data2)
print(target2)

torch.Size([15439, 1000])
torch.Size([15439])
tensor([[157., 166., 172.,  ..., 192., 188., 183.],
        [166., 172., 182.,  ..., 188., 183., 178.],
        [172., 182., 184.,  ..., 183., 178., 173.],
        ...,
        [147., 143., 134.,  ..., 138., 132., 127.],
        [143., 134., 123.,  ..., 132., 127., 123.],
        [134., 123., 122.,  ..., 127., 123., 109.]])
tensor([173., 169., 161.,  ..., 109., 112., 118.])


In [35]:
from torch.utils.data import random_split

In [36]:
# 定义数据集大小
dataset_size = len(allData)
print(dataset_size)
# 计算划分的数据集大小
train_size = int(0.8 * dataset_size)
test_size = dataset_size - train_size
train_dataset, test_dataset = random_split(allData, [train_size, test_size])
print(len(train_dataset))
print(len(test_dataset))

226
180
46


In [37]:
train_loader = DataLoader(allData, batch_size=32, shuffle=False)
# test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [38]:
for inputs, targets in train_loader:
    print(inputs.shape)
    print(targets.shape)

# for inputs, targets in test_loader:
#     print(inputs.shape)
    # 在这里你可以对inputs和targets进行操作，它们已经被转换为张量

torch.Size([32, 15439, 1000])
torch.Size([32, 15439])
torch.Size([32, 15439, 1000])
torch.Size([32, 15439])
torch.Size([32, 15439, 1000])
torch.Size([32, 15439])
torch.Size([32, 15439, 1000])
torch.Size([32, 15439])
torch.Size([32, 15439, 1000])
torch.Size([32, 15439])
torch.Size([32, 15439, 1000])
torch.Size([32, 15439])
torch.Size([32, 15439, 1000])
torch.Size([32, 15439])
torch.Size([2, 15439, 1000])
torch.Size([2, 15439])


### 改动dataset

In [3]:
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
import numpy as np
import torch
data_path = r"./Aleppo2017_processed.csv"
class myData(Dataset):
    def __init__(self):
        self.df = pd.read_csv(data_path,encoding='utf-8')
        # 将所有被试的特征放入data_feature,将所有被试的标签放入data_targets
        
        self.all_data_feature = []
        self.all_data_target = []
        self.data_feature = []
        self.data_target = []
        self.ID = self.df['id'].value_counts().index
        self.all_data_feat = torch.zeros(len(self.ID[:10]),15439,7)    #bs,seq,feat_size
        seq = 1000    #这个也算超参数之一
        for i in self.ID[:10]:
            self.data = self.df[self.df['id'] == i].copy()
            self.data = self.data.sort_values(by='time')
            # 时间为行索引
            self.data.index = self.data['time']

            # 设置标签
            self.data['target'] = self.data['gl'].shift(-1)
            self.data.dropna()  # 使用了shift函数，在最后必然是有缺失值的，这里去掉缺失值所在行
            self.data = self.data[['gl', 'target']].astype(np.float32)  # 修改数据类型
            self.data = self.data[:16439]    #所有受试者中血糖数据个数最小 为16439

            # 划分数据窗口
           
            for index in range(len(self.data) - seq):
                # 构建特征集
                self.data_feature.append(self.data['gl'][index: index + seq].values)
                # 构建target集
                self.data_target.append(self.data['target'].iloc[index + seq])
            self.all_data_feature.append(self.data_feature)
            self.all_data_target.append(self.data_target)
            self.data_feature = []
            self.data_target = []
            print("id为 {}的受试者的数据窗口加载完成".format(i))
        #将数据类型转化为torch.Tensor
        self.all_data_feature = torch.tensor(self.all_data_feature)
        self.all_data_target = torch.tensor(self.all_data_target)
        
        #每个窗口有x个特征   
        #计算每个窗口的mean
        self.all_data_feat[:,:,0] = torch.mean(self.all_data_feature,dim=2)
        #计算每个窗口的max
        self.all_data_feat[:,:,1] = torch.max(self.all_data_feature,dim=2)[0]
        #计算每个窗口的min
        self.all_data_feat[:,:,2] = torch.min(self.all_data_feature,dim=2)[0]
        #计算每个窗口的标准差
        self.all_data_feat[:,:,3] = torch.std(self.all_data_feature,dim=2)
        
        #计算每个窗口的斜率
#         x = torch.arange(1000, dtype=torch.float32)
#         self.all_data_feat[:,:,4] = torch.plotfit(x,self.all_data_feature,deg=1)[0]

        #计算每个窗口的四分位数
        self.all_data_feat[:,:,4] = torch.quantile(self.all_data_feature, 0.25, dim=2)
        self.all_data_feat[:,:,5] = torch.quantile(self.all_data_feature, 0.5, dim=2)
        self.all_data_feat[:,:,6] = torch.quantile(self.all_data_feature, 0.75, dim=2)
        #计算每个窗口的时间特征
        #pass

    def __getitem__(self,idx):
        datas = self.all_data_feat[idx]
        targets = self.all_data_target[idx]
        return datas, targets

    def __len__(self):
        return len(self.ID[:10])

allData = myData()
# 自动调用__getitem__( )

id为 263的受试者的数据窗口加载完成
id为 77的受试者的数据窗口加载完成
id为 193的受试者的数据窗口加载完成
id为 277的受试者的数据窗口加载完成
id为 229的受试者的数据窗口加载完成
id为 245的受试者的数据窗口加载完成
id为 155的受试者的数据窗口加载完成
id为 251的受试者的数据窗口加载完成
id为 111的受试者的数据窗口加载完成
id为 164的受试者的数据窗口加载完成


In [5]:
data1,target1 = allData[0]
print(data1)
print(data1.shape)
print(target1)
print(target1.shape)

tensor([[192.4980, 344.0000,  52.0000,  ..., 142.0000, 198.0000, 242.0000],
        [192.6570, 344.0000,  52.0000,  ..., 142.0000, 198.0000, 242.0000],
        [192.8150, 344.0000,  52.0000,  ..., 142.7500, 198.5000, 242.0000],
        ...,
        [162.9530, 398.0000,  45.0000,  ..., 116.7500, 156.0000, 209.0000],
        [162.9440, 398.0000,  45.0000,  ..., 116.7500, 156.0000, 209.0000],
        [162.9320, 398.0000,  45.0000,  ..., 116.0000, 156.0000, 209.0000]])
torch.Size([15439, 7])
tensor([245., 248., 249.,  ..., 116., 118., 127.])
torch.Size([15439])


In [7]:
from torch.utils.data import random_split, DataLoader
dataset_size = len(allData)

# 计算划分的数据集大小
train_size = int(0.8 * dataset_size)
test_size = dataset_size - train_size

# 使用 random_split 方法来随机划分数据集
train_dataset, test_dataset = random_split(allData, [train_size, test_size])
print("训练集大小：", len(train_dataset))
print("测试集大小", len(test_dataset))

训练集大小： 8
测试集大小 2


In [10]:
batch_size =2
# train = TensorDataset(trainX,trainY)
# test = TensorDataset(testX,testY)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [11]:
for datas, targets in train_loader:
    print(datas.shape)
    print(targets.shape)

torch.Size([2, 15439, 7])
torch.Size([2, 15439])
torch.Size([2, 15439, 7])
torch.Size([2, 15439])
torch.Size([2, 15439, 7])
torch.Size([2, 15439])
torch.Size([2, 15439, 7])
torch.Size([2, 15439])



## 先将每个受试者的数据提取，安装时间排序再分段，每1000点为一段

### for example  第一个受试者的数据窗口

### 第一窗[0,1,2,3,4,......999] target = 1000
### 第二窗[1,2,3,4,5,......1000] target = 1001
### 第三窗[2,3,4,5,6,......1001] target = 1002
### 第四窗[3,4,5,6,7,......1002] target = 1003

#### 那data_feature 内的数据为：
[第一个受试者的数据：
[0,1,2,3,4,......999],    #targets:[1000]
[1,2,3,4,5,......1000],    #targets:[1001]
[2,3,4,5,6,......1001],    #targets:[1002]
[3,4,5,6,7,......1002],    #targets:[1003]
第二个受试者的数据：
[0,1,2,3,4,......999],
[1,2,3,4,5,......1000],
[2,3,4,5,6,......1001],
[3,4,5,6,7,......1002]]

#### 对每个窗口提取特征
[第一个受试者的数据：
[0,1,2,3,4,......999],  ==>(特征提取)    #feature:[均值, 标准差, 方差，最大值，最小值，四分位数]  target = 1000
[1,2,3,4,5,......1000], ==>(特征提取)    #feature:[均值, 标准差, 方差，最大值，最小值，四分位数]  target = 1001
[2,3,4,5,6,......1001], ==>(特征提取)    #feature:[均值, 标准差, 方差，最大值，最小值，四分位数]  target = 1002
[3,4,5,6,7,......1002], ==>(特征提取)    #feature:[均值, 标准差, 方差，最大值，最小值，四分位数]  target = 1003
第二个受试者的数据：
[0,1,2,3,4,......999],
[1,2,3,4,5,......1000],
[2,3,4,5,6,......1001],
[3,4,5,6,7,......1002]]

#### 对每个窗口提取特征
[第一个受试者的数据：
[0,1,2,3,4,......999],  ==>(特征提取)    #feature:[均值, 标准差, 方差，最大值，最小值，四分位数]
[1,2,3,4,5,......1000], ==>(特征提取)    #feature:[均值, 标准差, 方差，最大值，最小值，四分位数]
[2,3,4,5,6,......1001], ==>(特征提取)    #feature:[均值, 标准差, 方差，最大值，最小值，四分位数]
[3,4,5,6,7,......1002],  ==>(特征提取)   #feature:[均值, 标准差, 方差，最大值，最小值，四分位数]
[4,5,6,7,8,......1003],  ==>(特征提取)   #feature:[均值, 标准差, 方差，最大值，最小值，四分位数]
[5,6,7,8,9,......1004],  ==>(特征提取)   #feature:[均值, 标准差, 方差，最大值，最小值，四分位数]
[6,7,8,9,10,......1005],  ==>(特征提取)   #feature:[均值, 标准差, 方差，最大值，最小值，四分位数]
[7,8,9,10,11,......1006], ==>(特征提取)   #feature:[均值, 标准差, 方差，最大值，最小值，四分位数] 
[8,9,10,11,12,......1007],==>(特征提取)   #feature:[均值, 标准差, 方差，最大值，最小值，四分位数]
[9,10,11,12,13......1008],==>(特征提取)   #feature:[均值, 标准差, 方差，最大值，最小值，四分位数],target = 1009]
<!-- 以此类推 -->

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv(r"./Aleppo2017_processed.csv",encoding='utf-8')
print(df.head())

    id                 time     gl
0  183  2015-05-16 05:35:41  162.0
1  183  2015-05-16 05:30:41  164.0
2  183  2015-05-16 05:25:41  168.0
3  183  2015-05-16 05:20:41  169.0
4  183  2015-05-16 05:15:41  170.0


In [4]:
import torch
from torch import nn
device = torch.device("cuda:0")
class myLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(myLSTM, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.num_layers = num_layers

        # Building your LSTM
        # batch_first=True causes input/output tensors to be of shape (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)

        # Readout layer 在LSTM后再加一个全连接层，因为是回归问题，所以不能在线性层后加激活函数
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_().to(device)
        # 这里x.size(0)就是batch_size

        # Initialize cell state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_().to(device)

        # One time step
        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        out = self.fc(hn)

        return out

In [69]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split


data_path = "./Aleppo2017_processed.csv"
class myData(Dataset):
    def __init__(self):
        self.df = pd.read_csv(data_path, encoding='utf-8')
        
        self.data_feature = []
        self.data_target = []
        self.ID = self.df['id'].value_counts().index
        # self.all_data_feat = torch.zeros(len(self.ID[:100]), 15439, 7)  # bs,seq,feat_size
        seq = 288  # 这个也算超参数之一
        for i in self.ID[:10]:
            self.data = self.df[self.df['id'] == i].copy()
            self.data = self.data.sort_values(by='time')
            # 时间为行索引
            self.data.index = self.data['time']

            # 设置标签
            self.data['target'] = self.data['gl'].shift(-1)
            self.data.dropna()  # 使用了shift函数，在最后必然是有缺失值的，这里去掉缺失值所在行
            self.data = self.data[['gl', 'target']].astype(np.float32)  # 修改数据类型
            self.data = self.data[:2000]  # 所有受试者中血糖数据个数最小 为16439
            self.data = self.data.gl
            for j in range(2000 - seq):
                window = self.data.iloc[j:j+seq].values
                target = self.data.iloc[j+seq]
                self.data_feature.append(window)
                self.data_target.append(target)

        self.data_feature = np.array(self.data_feature)
        self.data_target = np.array(self.data_target)

        # 转换为 PyTorch 张量
        self.data_feature = torch.tensor(self.data_feature, dtype=torch.float32)
        self.data_target = torch.tensor(self.data_target, dtype=torch.float32)
        print(self.data_feature.shape)
        print(self.data_target.shape)

    def __getitem__(self, idx):
        datas = self.data_feature[idx]
        targets = self.data_target[idx]
        return datas, targets

    def __len__(self):
        return len(self.data_feature)


In [39]:
df = pd.read_csv(data_path, encoding='utf-8')

In [41]:
ID = df['id'].value_counts().index
ID

Index([263,  77, 193, 277, 229, 245, 155, 251, 111, 164,
       ...
       128, 264, 186, 162, 249,  39,  52, 266, 223, 289],
      dtype='int64', name='id', length=226)

In [56]:
i = 183
data = df[df['id'] == i].copy()
data = data.sort_values(by='time')
# 时间为行索引
data.index = data['time']

# 设置标签
data['target'] = data['gl'].shift(-1)
data.dropna()  # 使用了shift函数，在最后必然是有缺失值的，这里去掉缺失值所在行
data = data[['gl', 'target']].astype(np.float32)  # 修改数据类型
data = data[:2000]  # 所有受试者中血糖数据个数最小 为16439
print(data)
data = data.gl
print(data.iloc[0:10].values)
print(type(data.iloc[0:10].values))
print(data[10])
print(type(data[10]))

                        gl  target
time                              
2015-04-21 23:17:27  187.0   189.0
2015-04-21 23:22:27  189.0   189.0
2015-04-21 23:27:27  189.0   189.0
2015-04-21 23:32:27  189.0   185.0
2015-04-21 23:37:27  185.0   183.0
...                    ...     ...
2015-04-29 00:46:58  170.0   167.0
2015-04-29 00:51:56  167.0   171.0
2015-04-29 00:56:56  171.0   171.0
2015-04-29 01:01:57  171.0   170.0
2015-04-29 01:06:56  170.0   170.0

[2000 rows x 2 columns]
[187. 189. 189. 189. 185. 183. 183. 185. 185. 186.]
<class 'numpy.ndarray'>
190.0
<class 'numpy.float32'>


In [70]:
allData = myData()
# 自动调用__getitem__( )
datas, targets = allData[0]
datas.shape

torch.Size([17120, 288])
torch.Size([17120])


torch.Size([288])

In [64]:
data1, target1 = allData[1]
print(data1.shape)

torch.Size([288, 2])


In [28]:
# 5、划分训练集和测试集
# 定义数据集大小
dataset_size = len(allData)

# 计算划分的数据集大小
train_size = int(0.8 * dataset_size)
test_size = dataset_size - train_size

# 使用 random_split 方法来随机划分数据集
train_dataset, test_dataset = random_split(allData, [train_size, test_size])
print("训练集大小：", len(train_dataset))
print("测试集大小", len(test_dataset))

训练集大小： 8
测试集大小 2


In [31]:
batch_size = 1
# train = TensorDataset(trainX,trainY)
# test = TensorDataset(testX,testY)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [13]:
input_dim = 1  # 数据的特征数
hidden_dim = 32  # 隐藏层的神经元个数
num_layers = 2  # LSTM的层数
output_dim = 1  # 预测值的特征数
# （这是预测股票价格，所以这里特征数是1，如果预测一个单词，那么这里是one-hot向量的编码长度）

# 实例化模型
model = myLSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
if torch.cuda.is_available():
    allen = model.cuda()

In [14]:
# 定义优化器和损失函数
loss_fn = torch.nn.MSELoss(reduction='mean')  # 使用均方误差作为损失函数
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)  # 使用Adam优化算法
if torch.cuda.is_available():
    loss_fn = loss_fn.cuda()

In [15]:
# 设定数据遍历次数
num_epochs = 300
# 记录训练次数
total_train_step = 0
# 记录测试次数
total_test_step = 0

In [32]:
for data in train_loader:
    trains, targets = data
    if torch.cuda.is_available():
        trains = trains.cuda()
        targets = targets.cuda()
    print(trains.shape)
    print(targets.shape)
    y_train_pred = model(trains)

torch.Size([1, 1712, 288])
torch.Size([1, 1712])


RuntimeError: input.size(-1) must be equal to input_size. Expected 6, got 288